# Machine Learning for Sepsis Pathway 

Setup Libraries and plotting defaults. 

In [1]:
import sys
import os
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
#import statsmodels.formula.api as smf
#from statsmodels.stats.anova import anova_lm


import re
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from pandasql import sqldf
import sqlalchemy
import pyodbc

import dfply as dfp
from dfply import *

import datetime
from datetime import *

import pyodbc
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics         import balanced_accuracy_score, precision_score, classification_report
from sklearn.metrics         import recall_score, f1_score, make_scorer, cohen_kappa_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import sklearn

from sklearn import model_selection
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier

import forestci as fci

from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler, PowerTransformer, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, FunctionTransformer, PowerTransformer
from sklearn.metrics import r2_score, classification_report, confusion_matrix, roc_auc_score, accuracy_score, balanced_accuracy_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
#from statsmodels.stats.outliers_influence import variance_inflation_factor
#from statsmodels.tools.tools import add_constant
import xgboost as xgb
from imblearn.over_sampling import SMOTENC
n_est=55

from hyperopt.pyll import scope
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK

pd.set_option('display.max_columns', None)


import Utilities
import importlib
importlib.reload(Utilities)

import MLUtilities
importlib.reload(MLUtilities)

from matplotlib.colors import to_rgba

plt.style.use('./CEIH.mplstyle')
color_dict = {0: to_rgba('#32C0D2', 1),
                1: to_rgba("#E0B165", 1),}

cmap_blended = sns.blend_palette(["#ADE6ED","#70D3E0","#32C0D2","#289AA8","#307078"], as_cmap=True)


import math
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1],idx-1
    else:
        return array[idx],idx
    
OtherValues = {'TriageCategory_2':'Triage Category 2', 'SoBFlag': 'Shortness of Breath', 'TriageCategory_4':'Triage Category 4',
               'TriageCategory_1':'Triage Category 1', 'TriageCategory_5':'Triage Category 5', 'TriageCategory_3':'Triage Category 3',
             'FirstGCSScoreAdult':'Glasgow Coma Scale',
               'FirstUrinalysisBlood': 'Urinalysis Blood', 'IndigenousStatusDescription_Not Aboriginal-TSI':  'Non Indiginous',
               'IndigenousStatusDescription_Not Stated':  'Indiginous Status not stated', 'FirstUrinalysisLeukocytes':'Urinalysis Leukocytes',
               'FirstPulseRateBPM':'Pulse Rate', 'FirstRespiration':'Respiration Rate', 'FirstSpO2':'O2 Saturation (%)',  'Gender_Male':'Sex - Male',
               'FirstLevelofConsciousness':'Level of Consciousness', 'IndigenousStatusDescription_Aboriginal and TSI':'Aboriginal and TSI',
               'IndigenousStatusDescription_TSI':'TSI','Gender_Indeterminate':'Sex - Indeterminate', 'Gender_Unknown':'Sex - Unknown',
               'FirstO2Flow':'O2 Flow '
               }


plotSettings = {
        #  ( bins, xmin, xmax, log/linear)
        'AGEONADMISSION':(50,20,110,'linear','Age (y)', False, 0,0),
        
        
        'FirstBloodGlucose':(50,0,100,'log','Blood Glucose [mmol/L]', False,0, 0),
        'FirstTemperatureDegreesC':(50,30,45,'log', r"Temperature [$^\circ$C]", True,35.5,38.1),
                'FirstWeightKg':(50,50,150,'log','Weight [kg]', False, 0,0),

        
        'FirstPainAssessment': (11,-0.5,10.5,'log','Pain Assessment', False, 0,0),
        'FirstBPSystolic': (50,50,250,'log', 'BP Systolic [mm Hg]',True, 100,170 ),
        'FirstBPDiastolic': (50,0,200,'log', 'BP Diastolic [mm Hg]', False,0,0),
        'FirstEstimatedGlomerularFiltrationRate': (50,0,100,'log',r"Estimated Glomerular Filtration Rate [mL/min/1.73m$^{2}$]", True, 60, 100),
        'FirstCreatinine': (50,0,800,'log',r"Creatinine - Serum [$\mu$mol/L]", True, 45, 110),
        'FirstAlbumin': (60,0,60,'log','Albumin  Level [g/L]', True, 30, 48),
        'FirstTotalBilirubin': (60,0,100,'log',r"Total Bilirubin Level [$\mu$mol/L]", True, 2, 24 ),
        'FirstAlkalinePhosphatase': (60,0,800,'log','Alkaline Phosphatase Level [U/L]',True, 30,110),
        'FirstAlanineAminotransferase': (60,0,700,'log','Alanine Aminotransferase Level [U/L]', True,0,55),
        'FirstAspartateAminotransferase': (60,0,700,'log','Aspartate Aminotransferase Level [U/L]', True, 0,45),
        'FirstGammaGlutamylTransferase': (60,0,700,'log','Gamma Glutamyl Transferase Level [U/L]', True, 0, 60),
        'FirstLactateDehydrogenase': (60,0,1200,'log','Lactate Dehydrogenase [U/L]', True, 120, 250),
        'FirstHaemoglobin': (50,10, 220,'log','Haemoglobin [g/L]', True, 115, 175),
        'FirstWhiteCellCount': (50,0, 50,'log',r"White Cell Count [$\times 10^{9}$/L]",True, 4,11),
        'FirstPlateletCount': (50,0, 1000,'log',r"Platelet Count [$\times 10^{9}$/L]",True,150,500),
        'FirstNeutrophils': (50,0, 50,'log',r"Absolute Neutrophil Count [$\times 10^{9}$/L]",True,1.80,7.50),
        'FirstDDimer': (40,0, 20,'log',r"D-Dimer [mg/L]",True,0,0.79),
        'FirstCreactiveprotein': (50,0, 600,'log',r"C-Reactive Protein [mg/L]",True, 0,8),
        'FirstTroponinT': (50,0, 600,'log',r"Troponin T Level [mg/L]",True,0,16),
        'FirstNTproBNP': (50,0, 40000,'log',r"NT-pro Brain Natriuretic Peptide [mg/L]",True,0,124),
        
        'FirstAnionGapVenous': (50,0, 50,'log',r"Anion Gap Venous [mmol/L]", True, 7, 17),
        'FirstAnionGapArterial': (50,0, 50,'log',r"Anion Gap Arterial [mmol/L]",  True, 7, 17),
        'FirstBaseExcessVenous': (50,-30, 30,'log',r"Base Excess Venous [mmol/L]", True, -3, 3),
        'FirstBaseExcessArterial': (50,-30, 30,'log',r"Base Excess Arterial [mmol/L]", True, -3, 3),
        'FirstBilirubinVenous': (60,0, 60,'log',r"Bilirubin Venous [$\mu$mol/L]", True, 2,24),
        'FirstBilirubinArterial': (60,0, 60,'log',r"Bilirubin Arterial [$\mu$mol/L]", True, 2,24),
        'FirstCarboxyhaemoglobinVenous': (50,0, 20,'log',r"Carboxyhaemoglobin Venous [%]", True, 0.3, 1.8),
        'FirstCarboxyhaemoglobinArterial': (50,0, 20,'log',r"Carboxyhaemoglobin Arterial [%]", True, 0.3, 1.8),
        'FirstChlorideDirectVenous': (50,50, 150,'log',r"Chloride Direct Venous [mmol/L]",False, 100,109),
        'FirstChlorideDirectArterial': (50,50, 150,'log',r"Chloride Direct Arterial [mmol/L]", True, 100,109),
        'FirstCreatinineVenous':(50,0,500,'log',r"Creatinine Venous [$\mu$mol/L]", True, 50, 120),
        'FirstCreatinineArterial':(50,0,500,'log',r"Creatinine Arterial [$\mu$mol/L]", True, 50,120),
        'FirstGlucoseVenous':(50,0,30,'log',r"Glucose  Venous [mmol/L]", False,0,0),
        'FirstGlucoseArterial':(50,0,30,'log',r"Glucose  Arterial [mmol/L]", True,2.6,5.6),
        'FirstIonised Calcium Venous':(50,0,2,'log',r"Ionised Calcium Venous [mmol/L]", True, 1.1, 1.3),
        'FirstIonised Calcium Arterial':(50,0,2,'log',r"Ionised Calcium Arterial [mmol/L]", True, 1.1, 1.3),
        'FirstLactateVenous':(50,0,30,'log',r"Lactate Venous [mmol/L]", True, 0.2, 2.0),
        'FirstLactateArterial':(50,0,30,'log',r"Lactate Arterial [mmol/L]", True, 0.2, 2.0),
        'FirstMethaemoglobinVenous': (20,0, 3,'log',r"Methaemoglobin Venous [%]", True, 0.4, 1.2),
        'FirstMethaemoglobinArterial': (20,0, 3,'log',r"Methaemoglobin Arterial [%]", True, 0.2,0.6),
        'FirstOxygenSaturationVenous': (50,0, 100,'log',r"Oxygen Saturation Venous [%]", False, 0,0),
        'FirstOxygenSaturationArterial': (50,0, 100,'log',r"Oxygen Saturation Arterial [%]", True,95, 99),
        'FirstOxyhaemoglobinVenous': (50,0, 100,'log',r"Oxyhaemoglobin Venous [%]", False, 0,0),
        'FirstOxyhaemoglobinArterial': (50,0, 100,'log',r"Oxyhaemoglobin Arterial [%]", False, 0,0),
        'FirstReducedHaemoglobinVenous': (50,0, 100,'log',r"Reduced Haemoglobin Venous [%]", False, 0,0),
        'FirstReducedHaemoglobinArterial': (50,0, 100,'log',r"Reduced Haemoglobin Arterial [%]", False, 0,0),
        'FirstTotalHaemoglobinVenous': (50,10, 220,'log','Total Haemoglobin Venous [g/L]', True, 115,180),
        'FirstTotalHaemoglobinArterial': (50,10, 220,'log','Total Haemoglobin Arterial [g/L]', True, 115,180), 
        'FirstpCO2Venous': (50,0,150,'log', 'pCO2 Venous [mm Hg]', True,41,51),
        'FirstpCO2Arterial': (50,0,150,'log', 'pCO2 Arterial [mm Hg]', True, 35, 45 ),
        'FirstpO2Venous': (50,0,200,'log', 'pO2 Venous [mm Hg]', True, 25,40),
        'FirstpO2Arterial': (50,0,200,'log', 'pO2 Arterial [mm Hg]', True,67, 108),
        'FirstpHVenous': (50,6.8,7.8,'log', 'pH Venous', True, 7.32,7.42),
        'FirstpHArterial': (50,6.8,7.8,'log', 'pH Arterial', True, 7.36, 7.44),
        'FirstPotassiumDirectVenous':(50,0,10,'log',r"Potassium Direct Venous [mmol/L]", False, 0,0),
        'FirstPotassiumDirectArterial':(50,0,10,'log',r"Potassium Direct Arterial [mmol/L]", True, 3.1, 4.2),
        'FirstSodiumDirectVenous':(50,100,180,'log',r"Sodium Direct Venous [mmol/L]", False, 0,0),
        'FirstSodiumDirectArterial':(50,100,180,'log',r"Sodium Direct Arterial [mmol/L]", True, 137, 145),}


Failed to import duecredit due to No module named 'duecredit'
c:\Users\ibertr02\venvs\ceihml\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (15.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
GenMedList = ['LMH-LGMED','LMH-LAMU',
              "MPH-MGMED", 
              "FMC-GENMED",
              "NHS-GENMED",
              "RAH-GENMED",
              "QEH-GENMED", 'QEH-GM-AMU'
              ]

# Settings for Machine Learning Model 

Modified code to exclude all < 18 yo patients.

Models based on first set of vital signs taken in ED. 

- DataSet 01: BP Systolic, Level of Consciousness, Pulse Rate (BPM), Respiration,  SpO2,  Temperature (Degrees C), O2 Flow
    + 01_a : Require age 65+
    + 01_b : 18 to 65 
- DataSet 02: DataSet 1 excluding O2 Flow 
    + 02_a : Require age 65+
    + 02_b : 18 to 65 
- DataSet 11: DataSet 1 plus Gender 
    + 11_a : Require age 65+
    + 11_b : 18 to 65 
- DataSet 12: DataSet 2 plus Gender 
    + 12_a : Require age 65+
    + 12_b : 18 to 65 
- DataSet 21: DataSet 1 plus Gender and Age
    + 21_a : Require age 65+
    + 21_b : 18 to 65 
- DataSet 22: DataSet 2 plus Gender and Age
    + 22_a : Require age 65+
    + 22_b : 18 to 65 

In [3]:
# Sepsis HeartFailure PE Pneumonia COPD UTI
# SepsisPneumonia
Diagnosis =   "Sepsis"
# Diagnosis = 'SepsisPneumonia'
# Diagnosis = 'Admission'
# Diagnosis = 'GenMed'

DiagnosisString=Diagnosis+'Flag'


# DataSet = 'DataSet3a'

# MLModel = 'randomforest'

print(Diagnosis,DiagnosisString)

Sepsis SepsisFlag


In [4]:
start_date =  '2023-01-01'  #dates[Facility][0]
end_date   =  '2024-01-01' #dates[Facility][1]

select_start_date = '2023-01-01'

DataReasons = "Training"

Data_Storage_File = 'DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_{}_{}_{}_Stage_01.pkl'.format(start_date,end_date,DataReasons)
print(Data_Storage_File)


with open(Data_Storage_File, 'rb') as file:
    Emergency_IPInfo_Complete_Numeric = pd.read_pickle(file)

DataFiles\Emergency_IPInfo_Complete_Numeric_Numeric_2023-01-01_2024-01-01_Training_Stage_01.pkl


In [5]:
Emergency_IPInfo_Complete_Numeric = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION>17].copy()

Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(0,17),'age_range']='0-17'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(18,33),'age_range']='18-33'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(34,48),'age_range']='34-48'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(49,64),'age_range']='49-64'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(65,78),'age_range']='65-78'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(79,98),'age_range']='79-98'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(98,120),'age_range']='98+'
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION.between(121,200),'age_range']='Unknown'

Emergency_IPInfo_Complete_Numeric['IndiginousFlag'] = 0  # np.where((Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isna()|Emergency_IPInfo_Complete_Numeric.IndiginousStatus=='Not Aboriginal-TSI'),0,1)
Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.IndiginousStatus.isin(['Aboriginal','Aboriginal and TSI', 'TSI']),'IndiginousFlag'] = 1


# DiagnosisString='SepsisFlag'

if Diagnosis == "SepsisPneumonia":
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.SepsisFlag==1)|(Emergency_IPInfo_Complete_Numeric.PneumoniaFlag==1), DiagnosisString] = 1

if Diagnosis == 'Admission':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>24), DiagnosisString] = 1
    
    if DataSet == 'DataSet5':
        Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
        Emergency_IPInfo_Complete_Numeric.loc[ (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1


if Diagnosis == 'GenMed':
    Emergency_IPInfo_Complete_Numeric[DiagnosisString] = 0
    for g in GenMedList:
        Emergency_IPInfo_Complete_Numeric.loc[(Emergency_IPInfo_Complete_Numeric.GROUPCODE.str.contains(g))  & (Emergency_IPInfo_Complete_Numeric['InitialIPLoS']>6), DiagnosisString] = 1
   


In [6]:
importlib.reload(MLUtilities)
Emergency_IPInfo_Complete_Numeric = MLUtilities.setNumeric(Emergency_IPInfo_Complete_Numeric)

In [7]:
numeric_transformer = Pipeline(steps=[
        ('scaler', MinMaxScaler()),  # StandardScaler #MinMaxScaler # RobustScaler
        # ('scaler', RobustScaler(quantile_range=((25.0,75.0)))),  # StandardScaler #MinMaxScaler # RobustScaler
        #('skew', FunctionTransformer(log_transform))
    ])

space4xgbsepsis = {
    'random_state': n_est, 
    'max_depth': hp.choice('max_depth', np.arange(1,30, 1, dtype=int)), 
    'n_estimators': hp.choice('n_estimators', np.arange(1, 200, 1, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),  # New 
    'subsample':hp.uniform('subsample', 0.5, 1.),     # New
     }

space4rf = {
    'random_state': n_est,
    'class_weight': 'balanced',
    'max_depth':  None, #    hp.choice('max_depth', np.arange(5, 30, 1, dtype=int)),
    #'max_features':    #hp.choice('max_features', np.arange(2, 10, 1, dtype=int)),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 500, 10, dtype=int))
}



In [8]:
def hyperparameter_tuning_rf_sepsis(trial_params):
        
    classifier = RandomForestClassifier(**trial_params)

    model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", classifier)
        ])
    model.fit(X_train, np.ravel(y_train,order="c"))
    predictions_valid = model.predict_proba(X_valid)
    ypreds = np.delete(predictions_valid,[0],1)
    
    fpr, tpr, thresholds = roc_curve(y_valid, ypreds)
    val,idx = find_nearest(tpr,0.85)
    threshold =   np.interp(0.85,tpr,thresholds)    #thresholds[idx]
    false_positive =  np.interp(0.85,tpr,fpr)      # fpr[idx]
    # redo predictions with new threshold 
    y_select_2 = np.where(ypreds<threshold,0,1)
    auc_score = roc_auc_score(y_valid, ypreds, multi_class="ovr", average="weighted")
    accuracy  =accuracy_score(y_valid, y_select_2)
    balanced_accuracy = balanced_accuracy_score (y_valid, y_select_2)
    recall  =recall_score(y_valid, y_select_2)
    precision = precision_score(y_valid, y_select_2)
    f1 = f1_score(y_valid, y_select_2)
    

    
    scores = {"falsePositive":false_positive,
              "accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "precision":precision,
              "f1":f1,
              "auc":auc_score,
              "threshold":threshold}

    return {'status': STATUS_OK, 'loss': false_positive, 'model': model, 'attachments':scores}


def hyperparameter_tuning_xgb_sepsis(trial_params):
            
    classifier = xgb.XGBClassifier(**trial_params)

    model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", classifier)
        ])
    
    model.fit(X_train, np.ravel(y_train,order="c"))
    predictions_valid = model.predict_proba(X_valid)
    ypreds = np.delete(predictions_valid,[0],1)
    
    fpr, tpr, thresholds = roc_curve(y_valid, ypreds)
    val,idx = find_nearest(tpr,0.85)
    threshold =   np.interp(0.85,tpr,thresholds)    #thresholds[idx]
    false_positive =  np.interp(0.85,tpr,fpr)      # fpr[idx]
    # redo predictions with new threshold 
    y_select_2 = np.where(ypreds<threshold,0,1)
    auc_score = roc_auc_score(y_valid, ypreds, multi_class="ovr", average="weighted")
    accuracy  =accuracy_score(y_valid, y_select_2)
    balanced_accuracy = balanced_accuracy_score (y_valid, y_select_2)
    recall  =recall_score(y_valid, y_select_2)
    precision = precision_score(y_valid, y_select_2)
    f1 = f1_score(y_valid, y_select_2)
    


    scores = {"falsePositive":false_positive,
              "accuracy":accuracy,
              "precision":precision,
              "recall":recall,
              "precision":precision,
              "f1":f1,
              "auc":auc_score,
              "threshold":threshold}

    return {'status': STATUS_OK, 'loss': false_positive, 'model': model, 'attachments':scores}

In [12]:
for DataSet in ['DataSet01',#'DataSet01_a','DataSet01_b','DataSet02','DataSet02_a','DataSet02_b',
                'DataSet11',#'DataSet11_a','DataSet11_b','DataSet12','DataSet12_a','DataSet12_b',
                'DataSet21',#'DataSet21_a','DataSet21_b','DataSet22','DataSet22_a','DataSet22_b',]:
]:
    
    n_trials = 100
    
    AnalysisVariable =['FirstBPSystolic', 'FirstLevelofConsciousness',
       'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2',
       'FirstTemperatureDegreesC', DiagnosisString]

    if '1_' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if '01' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if '11' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if '21' in DataSet:
        AnalysisVariable.append('FirstO2Flow')
    if ('DataSet1' in DataSet):
        AnalysisVariable.append('GENDERCODE')
    if ('DataSet2' in DataSet):
        AnalysisVariable.append('GENDERCODE')
        AnalysisVariable.append('AGEONADMISSION')

        
    if '_a' in DataSet:
        print("check")
        df = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION>64].copy()
    elif '_b' in DataSet:
        df = Emergency_IPInfo_Complete_Numeric.loc[Emergency_IPInfo_Complete_Numeric.AGEONADMISSION<65].copy()
    else: 
        df = Emergency_IPInfo_Complete_Numeric.copy()
        
        
        
    df.name='{} diagnosis detected from full Diagnosis list, updated'.format(Diagnosis)
    importlib.reload(MLUtilities)
    df = MLUtilities.setDefaults(df)

    print("\n{}".format(DataSet))
    print("Diagnosis being investigated = {}".format(DiagnosisString))
    print("Total Number of Cases = {}".format(len(df.index)))
    print("Number of {} Cases ".format(DiagnosisString),len(df[df[DiagnosisString]==1]))
    print(AnalysisVariable)
    
    df_temp = df[AnalysisVariable].copy()
    
    if 'GENDERCODE' in AnalysisVariable:
        df_temp['GENDERCODE'] = df_temp['GENDERCODE'].astype('category')
    
    # SMOTE does not work if NAN is allowed so int instead of Int 

    int_features = df_temp.select_dtypes(
        include=[ 'int64', 'Int64', 'int32']).columns

    for key in int_features:
        df_temp[key] = df_temp[key].astype('int64')
    
    y = df_temp[[DiagnosisString]]
    X = df_temp[AnalysisVariable].drop(DiagnosisString, axis=1).copy()
    x_vars = X.columns
    
    skew_vars = []

    
    numeric_features = X.drop(skew_vars, axis=1).select_dtypes(
        include=['float64', 'int64']).columns

    categorical_features = X.select_dtypes(
        include=['category', 'object']).columns
    
    category_values = []
    for i in categorical_features:
        tmp = pd.unique(df_temp[i]).tolist()
        tmp.sort()
        print(i,tmp)
        category_values.append(tmp)
    
    categorical_transformer = Pipeline(steps=[
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore',categories=category_values   ))])  # OrdinalEncoder() #OneHotEncoder(drop='first')

    preprocessor = ColumnTransformer(
            transformers=[
                ('numeric', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, categorical_features)
            ])

    def pip_f(reg):
        pip = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', reg)
        ])
        return(pip)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)
    
    if len(categorical_features)>0:
        smote = SMOTENC(random_state=42, categorical_features=[X_train.columns.get_loc(c) for c in categorical_features])
        X_train, y_train= smote.fit_resample(X_train, y_train)
        
        
    # n_trials = 2
    
    trialsRandomForest = Trials()
    trialsXGBoost = Trials()


    best_params_XGB = fmin(fn = hyperparameter_tuning_xgb_sepsis,
                        space = space4xgbsepsis,
                        algo = tpe.suggest,
                        max_evals = n_trials*10, 
                        trials = trialsXGBoost
                        )

    best_params_RandomForest = fmin(fn=hyperparameter_tuning_rf_sepsis,
                        space=space4rf,
                        algo=tpe.suggest,
                        max_evals=n_trials,
                        trials=trialsRandomForest
                        )
    
    scoreRandomForest =  [t['result']['loss'] for t in trialsRandomForest.trials]
    best_model_RandomForest=trialsRandomForest.results[np.argmin([r['loss'] for r in trialsRandomForest.results])]['model']


    scoreXGBoost = [t['result']['loss'] for t in trialsXGBoost.trials]
    best_model_XGBoost=trialsXGBoost.results[np.argmin([r['loss'] for r in trialsXGBoost.results])]['model']
    
    all_cols = np.concatenate([numeric_features])

    if len(categorical_features)>0:
        new_cat_cols =best_model_RandomForest['preprocessor'].transformers_[1][1].named_steps['encoder'].get_feature_names_out(categorical_features)
        all_cols = np.concatenate([numeric_features,new_cat_cols])


    xgbValues = []
    varStrings = ["falsePositive",  "threshold","accuracy", "precision", "recall","precision","f1", "auc"]
    scores = {}
    i=0
    for i in range(n_trials*10):
        scores = {}
        # if i < 2:
        scores['max_depth'] = trialsXGBoost.trials[i]['misc']['vals']['max_depth'][0]+1
        scores['n_estimators'] = trialsXGBoost.trials[i]['misc']['vals']['n_estimators'][0]+1
        scores['learning_rate'] = trialsXGBoost.trials[i]['misc']['vals']['learning_rate'][0]+1
        scores['subsample'] = trialsXGBoost.trials[i]['misc']['vals']['subsample'][0]+1
        # scores['seed'] = trialsXGBoost.trials[i]['misc']['vals']['seed'][0]
            # 
            # 'n_estimators'
        for c in varStrings:
            # print(c)
            key = "ATTACH::{}::{}".format(i,c)
            scores[c] = trialsXGBoost.attachments[key]
            scores["i"] = i
        xgbValues.append(scores)
    # print (scores)
    trials_XGB_df = pd.DataFrame.from_dict(xgbValues)

    rfValues = []
    varStrings = ["falsePositive", "accuracy", "precision", "recall","precision","f1", "auc",]
    scores = {}
    i=0
    for i in range(n_trials):
        scores = {}
        # if i < 2:
        # scores['max_depth'] = trialsRandomForest.trials[i]['misc']['vals']['max_depth'][0]+1
        scores['n_estimators'] = trialsRandomForest.trials[i]['misc']['vals']['n_estimators'][0]+1

            # 
            # 'n_estimators'
        for c in varStrings:
            # print(c)
            key = "ATTACH::{}::{}".format(i,c)
            scores[c] = trialsRandomForest.attachments[key]
            scores["i"] = i
        rfValues.append(scores)

    trials_rf_df = pd.DataFrame.from_dict(rfValues)
    
    ObjectsToSaveRF = [best_model_RandomForest, best_model_XGBoost, all_cols, X_valid, y_valid,trials_rf_df,trials_XGB_df]

    # timestr =  str(datetime.datetime.now().strftime('%Y_%m_%d'))
    # Data_Storage_File = 'Results/GenMed_SepsisPathway_{}_{}_FeatureImportances.pkl'.format(DataSet,DiagnosisString)

    Data_Storage_File = 'Results/Nov_2024_{}_{}_BestFit_Models_Pathway_ED.pkl'.format(DiagnosisString,DataSet)
    print(Data_Storage_File)

    with open(Data_Storage_File, 'wb') as file:  
        pickle.dump(ObjectsToSaveRF, file)



DataSet01
Diagnosis being investigated = SepsisFlag
Total Number of Cases = 277021
Number of SepsisFlag Cases  2733
['FirstBPSystolic', 'FirstLevelofConsciousness', 'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2', 'FirstTemperatureDegreesC', 'SepsisFlag', 'FirstO2Flow']
100%|██████████| 100/100 [2:06:33<00:00, 75.94s/trial, best loss: 0.3845679872609517]  
Results/Nov_2024_SepsisFlag_DataSet01_BestFit_Models_Pathway_ED.pkl

DataSet11
Diagnosis being investigated = SepsisFlag
Total Number of Cases = 277021
Number of SepsisFlag Cases  2733
['FirstBPSystolic', 'FirstLevelofConsciousness', 'FirstPulseRateBPM', 'FirstRespiration', 'FirstSpO2', 'FirstTemperatureDegreesC', 'SepsisFlag', 'FirstO2Flow', 'GENDERCODE']
GENDERCODE ['Female', 'Indeterminate', 'Male', 'Unknown']
100%|██████████| 100/100 [8:37:33<00:00, 310.53s/trial, best loss: 0.3638149403915563] 
Results/Nov_2024_SepsisFlag_DataSet11_BestFit_Models_Pathway_ED.pkl

DataSet21
Diagnosis being investigated = SepsisFlag
Total Num

In [10]:
trialsXGBoost.results[1]['model'][1].get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.16248644728822462,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 29,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 181,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': 55,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.8958502308437364,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [16]:
all_cols = np.concatenate([numeric_features])

if len(categorical_features)>0:
    new_cat_cols =best_model_RandomForest['preprocessor'].transformers_[1][1].named_steps['encoder'].get_feature_names_out(categorical_features)
    all_cols = np.concatenate([numeric_features,new_cat_cols])

In [24]:
xgbValues = []
varStrings = ["falsePositive",  "threshold","accuracy", "precision", "recall","precision","f1", "auc"]
scores = {}
i=0
for i in range(n_trials*10):
    scores = {}
    # if i < 2:
    scores['max_depth'] = trialsXGBoost.trials[i]['misc']['vals']['max_depth'][0]+1
    scores['n_estimators'] = trialsXGBoost.trials[i]['misc']['vals']['n_estimators'][0]+1
    scores['learning_rate'] = trialsXGBoost.trials[i]['misc']['vals']['learning_rate'][0]+1
    scores['subsample'] = trialsXGBoost.trials[i]['misc']['vals']['subsample'][0]+1
    # scores['seed'] = trialsXGBoost.trials[i]['misc']['vals']['seed'][0]
        # 
        # 'n_estimators'
    for c in varStrings:
        # print(c)
        key = "ATTACH::{}::{}".format(i,c)
        scores[c] = trialsXGBoost.attachments[key]
        scores["i"] = i
    xgbValues.append(scores)
# print (scores)
trials_XGB_df = pd.DataFrame.from_dict(xgbValues)

rfValues = []
varStrings = ["falsePositive", "accuracy", "precision", "recall","precision","f1", "auc",]
scores = {}
i=0
for i in range(n_trials):
    scores = {}
    # if i < 2:
    # scores['max_depth'] = trialsRandomForest.trials[i]['misc']['vals']['max_depth'][0]+1
    scores['n_estimators'] = trialsRandomForest.trials[i]['misc']['vals']['n_estimators'][0]+1

        # 
        # 'n_estimators'
    for c in varStrings:
        # print(c)
        key = "ATTACH::{}::{}".format(i,c)
        scores[c] = trialsRandomForest.attachments[key]
        scores["i"] = i
    rfValues.append(scores)

trials_rf_df = pd.DataFrame.from_dict(rfValues)

In [25]:
ObjectsToSaveRF = [best_model_RandomForest, best_model_XGBoost, all_cols, X_valid, y_valid,trials_rf_df,trials_XGB_df]

# timestr =  str(datetime.datetime.now().strftime('%Y_%m_%d'))
# Data_Storage_File = 'Results/GenMed_SepsisPathway_{}_{}_FeatureImportances.pkl'.format(DataSet,DiagnosisString)

Data_Storage_File = 'Results/Nov_2024_{}_{}_BestFit_Models_Pathway_ED.pkl'.format(DiagnosisString,DataSet)
print(Data_Storage_File)

with open(Data_Storage_File, 'wb') as file:  
    pickle.dump(ObjectsToSaveRF, file)

Results/Nov_2024_SepsisFlag_DataSet11_a_BestFit_Models_Pathway_ED.pkl
